# NOTE before running the program replace the dst value to the your predict folder's path on line 136

In [40]:
import subprocess
import tkinter as tk
import os
from tkinter import filedialog
print(tk.TkVersion)
import re


from keras.models import load_model
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
import keras



import pandas as pd
import cv2
import matplotlib.pyplot as plt
import random
import numpy as np



m = tk.Tk(className="Diabetic Retinopathy PROJECT (use png files)")

text_var0 = tk.StringVar(m)
text_var0.set('filepath')
lr0 = tk.Label(m, textvariable=text_var0)
lr0.pack()

text_var1 = tk.StringVar(m)
text_var1.set('None')
lr1 = tk.Label(m, textvariable=text_var1)
lr1.pack()
IMG_SIZE = 512

# the preprocessing functions

def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
    #         print(img1.shape,img2.shape,img3.shape)
            img = np.stack([img1,img2,img3],axis=-1)
    #         print(img.shape)
        return img



def load_ben_color(path, sigmaX=10):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = crop_image_from_gray(image)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image=cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , sigmaX) ,-4 ,128)
        
    return image


# the predict function
def setval():
#     preprocessing
    sa = re.split(r"/", fr'{text_var0.get()}')[-1]
    path = fr"predict/{sa}"
    image = load_ben_color(path, sigmaX=30)
    cv2.imwrite(path, image)
    
#     predicting
    listing_pngs = os.listdir('predict/')
    data_lis = []
    for i in listing_pngs:
        if i.endswith('.png'):
            data_lis.append(i)
    testd = pd.DataFrame({"id_code": data_lis})
    print(testd)
    
    loaded_model = load_model('model2.h5')

    datagen = ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input)
    test_dat = datagen.flow_from_dataframe(testd, directory='predict/', x_col='id_code', y_col=None, target_size=(224,224), class_mode=None
                                        , classes=None, batch_size=1, shuffle=False)
    
    loadedpredicz = loaded_model.predict_generator(test_dat, steps=1)
    print(loadedpredicz.shape)
    print(loadedpredicz)
    print(loadedpredicz.argmax(axis=1)[0])
    out_vale = loadedpredicz.argmax(axis=1)[0]
    
    
#     changing value
    if out_vale==1:
        text_var1.set("Positive")
    else:
        text_var1.set("Negative")
    print("SETTING VALUE")

    
    # removing file
    sa = re.split(r"/", fr'{text_var0.get()}')[-1]
    os.remove(fr"predict/{sa}")
    print("File Removed!")
    text_var0.set("Null")


# value reset function
def resetval():
    text_var0.set("Null")
    text_var1.set("Null")
    print("value resetted")
#     print(text_var0.get())


# file open and copy/paste function
def fileopener():
    m.filename = filedialog.askopenfilename(initialdir="/", title="SELECT FILES")
    text_var0.set(m.filename)
    if (fr'{text_var0.get()}'==""):
        print("nothing")
    else: 
        print("OPENING FILES")
        src = text_var0.get()
        print(src)
        src = src.replace('/', '\\')
        src = fr'{src}'
    
    
        print(src)
#         change this value to your current predict folder's path
        dst = r'E:\donatella\tkinter_try_outs\predict'
        subprocess.call(['xcopy', src, dst])


b0 = tk.Button(m, text='OPEN FILE', width=50, command=fileopener)
b0.pack()

b1 = tk.Button(m, text='Predict', width=50, command=setval)
b1.pack()

b1 = tk.Button(m, text='RESET', width=50, command=resetval)
b1.pack()

m.mainloop()

8.6
OPENING FILES
E:/donatella/CNN Kag/diabetic retinopathy/train_images/00f6c1be5a33.png
E:\donatella\CNN Kag\diabetic retinopathy\train_images\00f6c1be5a33.png
            id_code
0  00f6c1be5a33.png
Found 1 validated image filenames.
(1, 2)
[[0.45944422 0.5405558 ]]
1
SETTING VALUE
File Removed!
value resetted


### functions planned

<p> 
    create button as planned<br>
    file path input button's function is to let a user choose a file mainly a pic then automatically copy/paste the picture to the predict folder(create new folder named predict)<br> 
    reset button should reset predict label to null<br>
    after predicting delete all files in the predict folder automatically
    
</p>

### INSTRUCTIONS

<h2>at first click the OPEN FILE and choose the image file must be a png<br>
    Then click the PREDICT button <br>
    if you delete the value of filepath or results click RESET button

</h2>

In [38]:
# src = r'D:/DOWNLOAD/copied.jpg'
# src = src.replace('/', '\\')
# dst = r'E:\donatella\tkinter_try_outs\predict'
# subprocess.call(['xcopy', src, dst])

In [37]:
# src

In [36]:
# a = 2
# b = fr"\thello\tworld{a}"
# print(b)